# Projet Apprentissage profond

## Génération aléatoire

Dans cette partie, on commence un proverbs et le modèle le finit avec ses propres mots.

### Git

In [2]:
!git init
!git remote add origin https://github.com/louis-thevenet/n7-deep-learning.git
#!git pull origin main
!git pull origin wip-rag # A changer avant de merge

Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 144 (delta 79), reused 81 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 2.63 MiB | 5.42 MiB/s, done.
Resolving deltas: 100% (79/79), done.
From https://github.com/louis-thevenet/n7-deep-learning
 * branch            wip-rag    -> FETCH_HEAD
 * [new branch]      wip-rag    -> origin/wip-rag


### Importation des données

Les données sont des proverbes en anglais extrait de site internet.

In [3]:
import make_dataset

all_proverbs = make_dataset.load_data()

In [4]:
for key in all_proverbs:
    print(f"{key}: {len(all_proverbs[key])} proverbs")
print(f"Total: {sum(len(all_proverbs[key]) for key in all_proverbs)} proverbs")
total_length = sum(len(p) for proverbs in all_proverbs.values() for p in proverbs)
print(f"Total length: {total_length} characters")

print("\n\nExamples of proverbs:")
key = list(all_proverbs.keys())[0]
for i in range(5):
    print(all_proverbs[key][i])


proverbs_db_only_english.txt: 2208 proverbs
proverbs_db.txt: 34142 proverbs
proverbs_digest.txt: 1000 proverbs
Total: 37350 proverbs
Total length: 1853527 characters


Examples of proverbs:
He that brings good news, knocks hard.
Anger and haste hinder good counsel.
Big thunder, little rain.
Romeo must die in order to save the love.
The point is plain as a pike staff.


In [5]:
selected_proverbs_groups = [
    "proverbs_db.txt",
    "proverbs_digest.txt"
]

proverbs = []
for group in selected_proverbs_groups:
    proverbs.extend(all_proverbs[group])

### Fine Tuning

In [6]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 863.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "facebook/opt-125m"
# model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

On crée un objet DataSet dans lequel on met nos proverbes non tokenisés.
Puis on les donne aux Tokenizer du modèle de base.

In [8]:
from datasets import Dataset
used_proverbs = proverbs[0:20000]
dataset = Dataset.from_dict({"text": used_proverbs})
print(dataset)
max_length = max([len(x) for x in used_proverbs])
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Dataset({
    features: ['text'],
    num_rows: 20000
})


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [9]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,534,208 || trainable%: 0.2349


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-9-70358b48a957>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,7.177400
20,6.750600
30,6.479500


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline

# Configurer un pipeline de génération
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "A"
results = generator(prompt, max_length=max_length, num_return_sequences=10, do_sample=True, temperature=0.7)

for i, result in enumerate(results):
    print(f"--- Génération {i+1} ---")
    print(result['generated_text'])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

--- Génération 1 ---
A good boy never goes off to school, while a bad boy comes home.
--- Génération 2 ---
A dead horse is a dead horse.
--- Génération 3 ---
A man's best friend is still good.
--- Génération 4 ---
A man who is not a good man is not a good man.
--- Génération 5 ---
A good dog is a shepherd, so is a shepherd dog.
--- Génération 6 ---
A man is worth nothing.
--- Génération 7 ---
A good time to visit a family member.
--- Génération 8 ---
A lot of people like to think 'one man's thing doesn't take any other.'
--- Génération 9 ---
A good woman cannot keep her mouth closed.
--- Génération 10 ---
A man who has a heart of stone will never get a wife.


## Génération avec un thème

### Importation des données

In [12]:
all_proverbs = make_dataset.load_data_with_topics()

for key in all_proverbs:
    print(f"{key}: {len(all_proverbs[key])} proverbs")
print(f"Total: {sum(len(all_proverbs[key]) for key in all_proverbs)} proverbs")
total_length = sum(len(p) for proverbs in all_proverbs.values() for p in proverbs)
print(f"Total length: {total_length} characters")

print("\n\nExamples of proverbs:")
key = list(all_proverbs.keys())[0]
for i in range(5):
    print(all_proverbs[key][i])

selected_proverbs_groups = [
    "proverbs.jsonl"
]

proverbs = []
for group in selected_proverbs_groups:
    proverbs.extend(all_proverbs[group])


proverbs.jsonl: 34177 proverbs
Total: 34177 proverbs
Total length: 3254638 characters


Examples of proverbs:
### topics : [Ability, Escape, Nine, Ten] 

### proverb:  Courage is ten, nine is the ability to escape.
### topics : [Ability, Age, Knowledge, Youth] 

### proverb:  If Only youth had knowledge and old age ability.
### topics : [Ability, Attempt] 

### proverb:  The attempt exceeds the ability of one, not of a multitude.
### topics : [Ability, Breath] 

### proverb:  Life is your ability to breath out every time you breathe in.
### topics : [Ability, Wealth] 

### proverb:  Ability is the poor man's wealth.


### Fine-tuning

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

On crée un objet DataSet dans lequel on met nos proverbes non tokenisés. Puis on les donne aux Tokenizer de opt-125m.

In [47]:
from datasets import Dataset
used_proverbs = proverbs[0:12000]
dataset = Dataset.from_dict({"text": used_proverbs})
print(dataset)
max_length = max([len(x) for x in used_proverbs])
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Dataset({
    features: ['text'],
    num_rows: 12000
})


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [48]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,534,208 || trainable%: 0.2349


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-49-70358b48a957>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,8.217300
20,7.330200
30,6.653300
40,6.173400
50,5.583200
60,4.663700
70,3.480500
80,1.810000
90,0.720300
100,0.354500


### Utilisation

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = """### topics : [Hate]


### proverb:"""

results = generator(prompt, max_length=max_length, num_return_sequences=10, do_sample=True, temperature=0.7)

for i, result in enumerate(results):
    print(result['generated_text'])